In [6]:
import urllib3
import json
import base64
import cv2
import os
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import initialize_app, storage
import datetime
import winsound as sd

accessKey = "4ac7ded1-af2c-4731-afad-d927a4749439"
# Firebase Key
if not firebase_admin._apps:
    
    cred = credentials.Certificate('C:/Users/jhg83/Videos/etri-7d90d-firebase-adminsdk-g4uic-3d44a91dc0.json') #Firebase 인증파일 위치
    default_app = firebase_admin.initialize_app(cred)
    
# Firestore db 
db = firestore.client()
collection_name = 'CCTVDATA'

#CCTV path 분석을 원하고자 하는 동영상 폴더 위치 넣어주세요,
vidpath = cv2.VideoCapture(r'C:\Users\jhg83\Videos\뉴스영상.mp4')

# 위급시 알림음
def beepsound():
    fr = 2000    
    du = 500    
    sd.Beep(fr, du) 

#beepsound()
    
    #객체 검출 api
class Object:
    def __init__(self,accesskey,path,time,doc_ref):
        self.accesskey = accesskey
        self.path = path
        self.time = time
        self.doc_ref = doc_ref
    def object_info(self):
        openApiURL = "http://aiopen.etri.re.kr:8000/ObjectDetect"
        accessKey = self.accesskey
        imageFilePath = self.path
        time = int(self.time)
        doc_ref = self.doc_ref
        
        #총 사람 숫자
        people_num = 0

        
        
        type = "jpg"
        file = open(imageFilePath, "rb")
        imageContents = base64.b64encode(file.read()).decode("utf8")
        file.close()
   
        requestJson = { 
            "argument": {
                "type": type,
                "file": imageContents
            }
        }
   
        http = urllib3.PoolManager()
        response = http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8" , "Authorization": accessKey},
            body=json.dumps(requestJson)
        )
        a = json.loads(response.data.decode('utf-8'))
        try:
            lenght = len(a['return_object']['data'])
        except:
            lenght = 0
        #각 객체별 for문
        number = 0
        for i in range(lenght):
            name = a['return_object']['data'][i]['class']
            confidence = a['return_object']['data'][i]['confidence']
            x_axis = int(a['return_object']['data'][i]['x'])
            y_axis = int(a['return_object']['data'][i]['y']) + 8
            width = int(a['return_object']['data'][i]['width'])
            height = int(a['return_object']['data'][i]['height'])
            print("Class = ",name,i,"\n","Confidence = ",confidence,"\n","X_axis =",x_axis,"\n","Y_axis =",y_axis,"\n","Width ="
                 ,width,"\n","Height =",height)
            
            # 사진 추출 및 저장
            image = cv2.imread(self.path)
            cropped_image = image[y_axis:y_axis+height, x_axis:x_axis+width]
            #" 해당 원하는 파일 위치"
            cv2.imwrite("C:/Users/jhg83/Videos/Test/time%d_%d_%s.jpg" %(time,number,name), cropped_image)

            imagename = "time%d_%d_%s" %(time,number,name)
            message_ref = doc_ref.collection("IMAGE").document(imagename)
            
            try:
                data = {
                'name' : name + str(i),
                'Confidence': confidence,
                'X' : x_axis,
                'Y' : y_axis,
                'Width' : width,
                'Height' : height
                }
                message_ref.set(data)
            except:
                print('No data')
                # 칼 발견시 따로 분류 나중에 car -> knife 로 수정
            if name == 'car': 
                dangerous_ref = db.collection('CCTVDATA').document("Dangerous_Objects")
                knife_ref = dangerous_ref.collection('Object').document(imagename)
                try:
                    data = {
                    'name' : name + str(i),
                    'Confidence': confidence,
                    'X' : x_axis,
                    'Y' : y_axis,
                    'Width' : width,
                    'Height' : height
                    }
                    knife_ref.set(data)
                except:
                    print('No data')                     
                
                
            if name =='person':
                status_check = Status("4ac7ded1-af2c-4731-afad-d927a4749439","C:/Users/jhg83/Videos/Test/time%d_%d_%s.jpg" 
                                      %(time,number,name),name,i,message_ref,imagename)
                status_check.status_info()
                people_num += 1
            
            # 100명이상 있을시 유의
            if people_num == 250:
                dangerous_ref = db.collection('CCTVDATA').document("Dangerous_condition")
                crash_ref = dangerous_ref.collection('CRASH_CONDITION').document(imagename)
                try:
                    data = {
                    'IMAGE_NAME' : crash_ref,
                    'People_num' : people_num
                    }
                    crash_ref.set(data)
                    print('Crash_condition_exist!, People num:',people_num)
                except:
                    print('No data')                       
            number += 1

            
     #사람 상태 이해 API
class Status:
    def __init__(self,accesskey,path,name,number,message_ref,imagename):
        self.accesskey = accesskey
        self.path = path
        self.name = name
        self.number = number
        self.message_ref = message_ref
        self.imagename = imagename
        
    def status_info(self):
        openApiURL = "http://aiopen.etri.re.kr:8000/HumanStatus"
        accesskey = self.accesskey
        imageFilepath = self.path
        name = self.name
        number = self.number
        message_ref = self.message_ref
        imagename = self.imagename
        type = "jpg"
        
        # 실시간 변수
        dt_now = datetime.datetime.now()
        file = open(imageFilePath, "rb")
        imageContents = base64.b64encode(file.read()).decode("utf8")
        file.close()
        requestJson = {
            "argument": {
              "type": type,
              "file": imageContents
            }
          }
  
        http = urllib3.PoolManager()
        response = http.request(
            "POST",
            openApiURL,
            headers={"Content-Type": "application/json; charset=UTF-8","Authorization": accessKey},
            body=json.dumps(requestJson)
          )
        a = json.loads(response.data.decode('utf-8'))
        try:
            lenght = len(a['return_object'][0]['data']) 
        except:
            lenght = 0
            
            
        try:
            name = name + str(number)
            status = a['return_object'][0]['data'][1]['class']
            confidence = a['return_object'][0]['data'][1]['confidence']
            x_axis = int(a['return_object'][0]['data'][1]['x'])
            y_axis = int(a['return_object'][0]['data'][1]['y']) 
            width = int(a['return_object'][0]['data'][1]['width'])
            height = int(a['return_object'][0]['data'][1]['height'])
            print("Status = ",status,"\n","Confidence = ",confidence,"\n","X_axis = ",x_axis,"\n","Y_axis = ",y_axis,"\n","Width = ",width,
                "\n","Height = ",height)          
            
            # Lying 상태 따로 분류 저장
            if status == 'Lying':
                emergency_ref = db.collection('CCTVDATA').document("EmergencyPerson")
                lying_ref = emergency_ref.collection('Lying').document(imagename) 

                try:
                    data = {
                    'name' : name,
                    'Status': status,
                    'Confidence': confidence,
                    'X' : x_axis,
                    'Y' : y_axis,
                    'Width' : width,
                    'Height' : height
                    }
                    lying_ref.set(data)
                except:
                    print('No data')                    
            # Crouch 상태 따로 분류 저장
            if status == 'Crouch':
                emergency_ref = db.collection('CCTVDATA').document("EmergencyPerson")
                crouch_ref = emergency_ref.collection('Crouch').document(imagename) 

                try:
                    data = {
                    'name' : name,
                    'Status': status,
                    'Confidence': confidence,
                    'X' : x_axis,
                    'Y' : y_axis,
                    'Width' : width,
                    'Height' : height
                    }
                    crouch_ref.set(data)
                except:
                    print('No data')   
                    
            if dt_now.hour > 23 and dt_now.hour < 6:
                if status == 'Lying' or status =='Crouch':
                    emergency_ref = db.collection('CCTVDATA').document("EmergencyPerson")
                    night_ref = emergency_ref.collection('Night').document(imagename)
                try:
                    data = {
                    'name' : name,
                    'Status': status,
                    'Confidence': confidence,
                    'X' : x_axis,
                    'Y' : y_axis,
                    'Width' : width,
                    'Height' : height
                    }
                    crouch_ref.set(data)
                except:
                    print('No data')                       
            
            try:
                data = {
                'name' : name,
                'Status': status,
                'Confidence': confidence,
                'X' : x_axis,
                'Y' : y_axis,
                'Width' : width,
                'Height' : height
                }
                message_ref.set(data)
            except:
                print('No data')
       
        except:
            print('Daily Limit exceeded')
    

count = 0

while(vidpath.isOpened()):
    ret, image = vidpath.read()
    image = cv2.resize(image, (960,540))
    # 5초당 이미지 추출
    if(int(vidpath.get(1)) % 300 == 0):
        
        # 추출된 이미지가 저장되는 경로 원하고자하는 파일 위치 넣어주세요 이미지 확장자는 jpg입니다
        cv2.imwrite("C:/Users/jhg83/Videos/time%d.jpg" % count, image)
        #Firebase document id 저장
        imageFilePath = "C:/Users/jhg83/Videos/time%d.jpg" % count
        docu_name = "time%d.jpg" % count
        document_id = docu_name
        doc_ref = db.collection(collection_name).document(document_id)
        print('--------------------------TIME : %d------------------------------' %count)
        object_check = Object("4ac7ded1-af2c-4731-afad-d927a4749439",imageFilePath,count,doc_ref)
        object_check.object_info()
        count += 1
        
        if count == 10:
            vidpath.release()
            print("fin")
    

--------------------------TIME : 0------------------------------
Class =  person 0 
 Confidence =  0.977754 
 X_axis = 158 
 Y_axis = 61 
 Width = 681 
 Height = 264
Status =  Crouch 
 Confidence =  0.7482889294624329 
 X_axis =  92 
 Y_axis =  18 
 Width =  747 
 Height =  418
--------------------------TIME : 1------------------------------
Class =  person 0 
 Confidence =  0.768703 
 X_axis = 241 
 Y_axis = 32 
 Width = 278 
 Height = 336
Status =  Crouch 
 Confidence =  0.7678225636482239 
 X_axis =  211 
 Y_axis =  12 
 Width =  342 
 Height =  356
--------------------------TIME : 2------------------------------
Class =  person 0 
 Confidence =  0.962987 
 X_axis = 577 
 Y_axis = 162 
 Width = 192 
 Height = 383
Status =  Standing 
 Confidence =  0.8817856907844543 
 X_axis =  7 
 Y_axis =  5 
 Width =  214 
 Height =  518
Class =  person 1 
 Confidence =  0.884632 
 X_axis = 147 
 Y_axis = 46 
 Width = 179 
 Height = 232
Status =  Standing 
 Confidence =  0.8817856907844543 
 X_ax